In [1]:
import csv
import os
import sys
import h5pyd
import xarray as xr
import numpy as np
import pandas as pd
import netCDF4 as nc
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree
from scipy import stats
import math

def haversine(lat1, lon1, lat2, lon2):
   # Convert latitude and longitude from degrees to radians
   lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

   dlat = lat2 - lat1
   dlon = lon2 - lon1
   a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
   c = 2 * math.asin(math.sqrt(a))

   # Radius of Earth in kilometers. Use 3956 for miles.
   r = 6371

   # Calculate the result
   distance = c * r
   return distance

print(sys.version)

# Create 2D array with file_id, water, and region values for file list
array = np.array([
[6678, 6, 1],[6687, 6, 1],[6772, 6, 1],[6783, 6, 1],[6950, 4, 1],[6963, 4, 0],
[7139, 6, 1],[7239, 2, 0],[7280, 6, 1],[7311, 6, 1],[7321, 6, 1],[7329, 5, 1],
[7361, 4, 1],[7414, 4, 1],[7555, 5, 1],[7571, 5, 1],[7585, 5, 1],[7704, 3, 1],
[7805, 5, 1],[7816, 5, 1],[7838, 5, 1],[7938, 3, 1],[7979, 3, 1],[7999, 1, 1],
[8000, 1, 1]])

# Set index to get file_id, water, and region values for each file
i=0

# Create empty list to store NSRDB file names
files = []

# Fill list with all possible file names
for j in range(8):
   ind = 1998+j
   files.append("/nrel/nsrdb/v3/nsrdb_"+str(ind)+".h5")

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

# Define the directory containing the files 
path = current_directory+"\\Data\\HADISDH_Humidity\\NC_Files\\"
print(path)

# Create an empty list to hold the dataframes 
dataframes = [] 

# Create datetime object for comparison
datetime_str = '1/1/98 00:00:00'

# Convert string to datetime
datetime_object = datetime.strptime(datetime_str, '%d/%m/%y %H:%M:%S')

# Loop through all filenames in filenames_only.csv 
with open("filenames_only_yearly.csv", mode='r') as file:
   csv_reader = csv.reader(file)
   for row in csv_reader:
      filename = row[0]
      print(filename)
      array_row = array[i]
      # Open the NetCDF file
      file = path+filename
      ds = xr.open_dataset(file)
      # Convert the dataset to a DataFrame
      df = ds.to_dataframe().reset_index()
      df_filtered = df[df['time'] >= datetime_object] # Remove data older than January 1, 1998
      # Fill NaN with linearly interpolated values
      df_filtered = df_filtered.interpolate()
      df_filtered.set_index('time', inplace = True)
      # Drop unneeded columns
      df_filtered = df_filtered.drop(['input_station_id','station_id','coordinate_length'], axis=1)
      df_filtered = df_filtered.drop(['relative_humidity', 'saturation_vapor_pressure', 'dewpoints'], axis=1)
      df_filtered = df_filtered.assign(file_id = int(array_row[0]))
      df_filtered = df_filtered.assign(region = int(array_row[1]))
      df_filtered = df_filtered.assign(water = int(array_row[2]))

      j=0

      # Create empty list for dataframes
      ghi_datasets = []
      wind_speed_datasets = []
      time_index_datasets = []

      # for loop to get data from NSRDB for each year
      for j in range(8):
         # Open the desired year of nsrdb data
         year = 1998 + j
         print("Year: "+str(year))
         files = "/nrel/nsrdb/v3/nsrdb_"+str(year)+".h5"
         f = h5pyd.File(files, 'r')
    
         #Lists attributes contained in file f
         list(f)

         # Locational information is stored in either 'meta' or 'coordinates'
         meta = pd.DataFrame(f['meta'][...])

         # Access the datasets for latitude and longitude
         latitude_dataset = meta['latitude']
         longitude_dataset = meta['longitude']

         # Read the data into numpy arrays
         latitude = latitude_dataset[:]
         longitude = longitude_dataset[:]

         # Print the extracted data
         print("Latitude:", latitude)
         print("Longitude:", longitude)

         indexes1 = []
         indexes2 = []
         values1 = []
         values2 = []

         #HADISDH geospatial coordinates
         latitude_h = df_filtered['latitude'].values[0]
         longitude_h = df_filtered['longitude'].values[0]

         #Range to look for NSRDB geospatial location
         latitude_min_NSRDB = latitude_h - 0.02
         latitude_max_NSRDB = latitude_h + 0.02
         longitude_min_NSRDB = longitude_h - 0.02
         longitude_max_NSRDB = longitude_h + 0.02

         print(latitude_min_NSRDB)
         print(latitude_max_NSRDB)
         print(longitude_min_NSRDB)
         print(longitude_max_NSRDB)

         # Extract datetime index for datasets
         time_index = pd.to_datetime(f['time_index'][...].astype(str))
         time_index # Temporal resolution is 30min
         print(time_index.shape[0])

         for index1 in range(2018392):
            value = latitude[index1]
            if(value < latitude_max_NSRDB and value > latitude_min_NSRDB):
               indexes1.append(index1)
               if value not in values1:
                  values1.append(value)

         for index2 in range(2018392):
            value = longitude[index2]
            if(value < longitude_max_NSRDB  and value > longitude_min_NSRDB):
               indexes2.append(index2)
               if value not in values2:
                  values2.append(value)

         # Find specific index for location from common index between latitude and longitude list
         common_indexes = []
         common_indexes = [value for value in indexes1 if value in indexes2]
         print(common_indexes)
         location_index = common_indexes[0]

         # Datasets are stored in a 2d array of time x location
         dset = f['ghi']
         print(dset.shape)
         print(dset)

         dset2 = f['wind_speed']
         print(dset2.shape)
         print(dset2)

         # Get GHI and wind speed data for specific location and times
         dset_filtered = pd.DataFrame(dset[0::2, location_index])
         dset2_filtered = pd.DataFrame(dset2[0::2, location_index])
         time_index_df = pd.DataFrame(time_index[0::2])

         ghi_datasets.append(dset_filtered)
         wind_speed_datasets.append(dset2_filtered)
         time_index_datasets.append(time_index_df)

         # Close the file
         f.close()

      # Concatenate all Dataframes
      combined_ghi_df = pd.concat(ghi_datasets, ignore_index=True)
      combined_wind_df = pd.concat(wind_speed_datasets, ignore_index=True)
      combined_time_df = pd.concat(time_index_datasets, ignore_index=True)

      # Now combined files contain all the data from GHI and wind speed files
      print(combined_ghi_df)
      print(combined_wind_df)
      print(combined_time_df)

      NSRDB_merged_df = pd.concat([combined_time_df, combined_ghi_df, combined_wind_df], axis=1)
      NSRDB_merged_df.columns = ['time', 'GHI', 'wind_speed']
      NSRDB_merged_df['NSRDB_latitude'] = round(latitude[location_index], 3)
      NSRDB_merged_df['NSRDB_longitude'] = round(longitude[location_index], 3)
      NSRDB_merged_df['distance_h_NSRDB'] = round(haversine(df_filtered['latitude'].values[0], df_filtered['longitude'].values[0], latitude[location_index], longitude[location_index]), 3)

      print(NSRDB_merged_df)
      print(NSRDB_merged_df.shape[0])

      df_filtered2 = df_filtered.reset_index()
      print(df_filtered2['time'])

      print(df_filtered2)

      for n in range(NSRDB_merged_df.shape[0]):
         datevalue = str(NSRDB_merged_df['time'].values[n])[:10]
         timevalue = str(NSRDB_merged_df['time'].values[n])[11:19]
         datetimevalue = datevalue + " " + timevalue
         NSRDB_merged_df.at[n, 'time'] = datetimevalue

      NSRDB_merged_df['time'] = pd.to_datetime(NSRDB_merged_df['time'])
      df_filtered2['time'] = pd.to_datetime(df_filtered2['time'])

      daily_NSRDB_df = NSRDB_merged_df.resample('D', on='time').mean()
      daily_df_filtered = df_filtered2.resample('D', on='time').mean()

      NSRDB_merged_df.set_index('time')
      df_filtered2.set_index('time')

      print(daily_NSRDB_df)
      print(daily_df_filtered)

      merged_df = pd.merge(daily_df_filtered, daily_NSRDB_df, on='time', how='outer')
      #merged_df['file_id'] = merged_df['file_id'].astype(int)
      #merged_df['water'] = merged_df['water'].astype(int)
      #merged_df['region'] = merged_df['region'].astype(int)
      print(merged_df)

      # Write DataFrame to CSV
      newfilename = "HADISDH_NSRDB_Merged_File_Yearly" + str(merged_df['file_id'].values[0]) + ".csv"
      merged_df.to_csv(newfilename, index=True)

      dataframes.append(merged_df)
      i += 1

# Concatenate all Dataframes
combined_df = pd.concat(dataframes, ignore_index=False)

# Now combined_df contains all the data from NC files
print(combined_df)

# Display the dataframe
print(combined_df.head())

# Write DataFrame to CSV
newfilename = "HADISDH_NSRDB_Merged_File_All_Data_Yearly.csv"
combined_df.to_csv(newfilename, index=True)

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
C:\Users\User
C:\Users\User\Data\HADISDH_Humidity\NC_Files\
hadisd.3.4.1.202411p_19310101-20241201_722069-53853_humidity.nc


C:\Users\User\AppData\Local\Temp\ipykernel_8428\81146084.py:87: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filtered = df_filtered.interpolate()


Year: 1998
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389    51.689999
2018390    51.770000
2018391    51.689999
Name: latitude, Length: 2018392, dtype: float32
Longitude: 0         -175.259995
1         -175.220001
2         -175.179993
3         -175.139999
4         -175.100006
              ...    
2018387    179.860001
2018388    179.860001
2018389    179.860001
2018390    179.860001
2018391    179.899994
Name: longitude, Length: 2018392, dtype: float32
30.373
30.413
-86.487
-86.447
17520
[927438]
(17520, 2018392)
<HDF5 dataset "ghi": shape (17520, 2018392), type "<i2">
(17520, 2018392)
<HDF5 dataset "wind_speed": shape (17520, 2018392), type "<i2">
Year: 1999
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389  

C:\Users\User\AppData\Local\Temp\ipykernel_8428\81146084.py:87: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filtered = df_filtered.interpolate()


Year: 1998
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389    51.689999
2018390    51.770000
2018391    51.689999
Name: latitude, Length: 2018392, dtype: float32
Longitude: 0         -175.259995
1         -175.220001
2         -175.179993
3         -175.139999
4         -175.100006
              ...    
2018387    179.860001
2018388    179.860001
2018389    179.860001
2018390    179.860001
2018391    179.899994
Name: longitude, Length: 2018392, dtype: float32
32.879
32.919000000000004
-80.06099999999999
-80.021
17520
[1063228]
(17520, 2018392)
<HDF5 dataset "ghi": shape (17520, 2018392), type "<i2">
(17520, 2018392)
<HDF5 dataset "wind_speed": shape (17520, 2018392), type "<i2">
Year: 1999
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
201838

...

Year: 1998
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389    51.689999
2018390    51.770000
2018391    51.689999
Name: latitude, Length: 2018392, dtype: float32
Longitude: 0         -175.259995
1         -175.220001
2         -175.179993
3         -175.139999
4         -175.100006
              ...    
2018387    179.860001
2018388    179.860001
2018389    179.860001
2018390    179.860001
2018391    179.899994
Name: longitude, Length: 2018392, dtype: float32
46.331999999999994
46.372
-104.28099999999999
-104.241
17520
[502715]
(17520, 2018392)
<HDF5 dataset "ghi": shape (17520, 2018392), type "<i2">
(17520, 2018392)
<HDF5 dataset "wind_speed": shape (17520, 2018392), type "<i2">
Year: 1999
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
20183

C:\Users\User\AppData\Local\Temp\ipykernel_8428\81146084.py:87: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filtered = df_filtered.interpolate()


Year: 1998
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389    51.689999
2018390    51.770000
2018391    51.689999
Name: latitude, Length: 2018392, dtype: float32
Longitude: 0         -175.259995
1         -175.220001
2         -175.179993
3         -175.139999
4         -175.100006
              ...    
2018387    179.860001
2018388    179.860001
2018389    179.860001
2018390    179.860001
2018391    179.899994
Name: longitude, Length: 2018392, dtype: float32
45.175999999999995
45.216
-123.154
-123.114
17520
[212086]
(17520, 2018392)
<HDF5 dataset "ghi": shape (17520, 2018392), type "<i2">
(17520, 2018392)
<HDF5 dataset "wind_speed": shape (17520, 2018392), type "<i2">
Year: 1999
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.84

C:\Users\User\AppData\Local\Temp\ipykernel_8428\81146084.py:87: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filtered = df_filtered.interpolate()


Year: 1998
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
2018388    51.849998
2018389    51.689999
2018390    51.770000
2018391    51.689999
Name: latitude, Length: 2018392, dtype: float32
Longitude: 0         -175.259995
1         -175.220001
2         -175.179993
3         -175.139999
4         -175.100006
              ...    
2018387    179.860001
2018388    179.860001
2018389    179.860001
2018390    179.860001
2018391    179.899994
Name: longitude, Length: 2018392, dtype: float32
45.806
45.846000000000004
-119.28099999999999
-119.241
17520
[219383]
(17520, 2018392)
<HDF5 dataset "ghi": shape (17520, 2018392), type "<i2">
(17520, 2018392)
<HDF5 dataset "wind_speed": shape (17520, 2018392), type "<i2">
Year: 1999
Latitude: 0         -19.990000
1         -19.990000
2         -19.990000
3         -19.990000
4         -19.990000
             ...    
2018387    51.810001
20183